In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
def kWh_to_kBTu(electricity):
    "Converts kilowatt-hours(kWh) to kilobtus(kBtu)"
    return electricity*3.4121

def therms_to_kBTu(gas):
    "Converts therms(thm) to kilobtus(kBtu)"
    return gas*99.976

def gallons_to_kBtu(oil):
    "Converts gallons to kilobtus(kBtu)"
    return oil*139

def eui(E, G, O, GSF): 
    """
    Calculates energy use intensity given energy uses and building gross square footage. 
    : param E - electricity use in kBtu
    : param G - gas use in kBtu
    : param O - natural oil use in kBtu
    : param GSF - building gross square footage
    """
    annual_energy_use = E + G + O
    return annual_energy_use/GSF
   

In [3]:
research_data = pd.read_csv('../data/research.csv', skiprows= 1)
predictions_data = pd.read_csv('../data/predictions.csv')
residential_data = pd.read_csv('../data/residential.csv', skiprows=1)

In [4]:
predictions_data

,Code,Address,ESPM Property Type,space@BU Property Type,2006Area
0,600,33 Harry Agganis Way,Residence Hall/Dormitory,Residential,400268.0
1,918,815 Albany Street,Multifamily Housing,Residential,102126.0
2,972,Peabody Hall (210 Riverway),Multifamily Housing,Residential,58637.0
3,973,Riverway House (160-162 Riverway),Residence Hall/Dormitory,Residential,31106.0
4,975,Campus Center and Student Residence (150 River...,Residence Hall/Dormitory,Residential,57107.0
5,969,37 Pilgrim Road,Residence Hall/Dormitory,Residential,NaN
6,974,154 Riverway,Residence Hall/Dormitory,Residential,NaN
7,608,610 Commonwealth Avenue,Laboratory,Research,170000.0


In [5]:
years = ['2006', '2015', '2016', '2017', '2018', '2019','2020','2021']
dataframes = []
for year in years:
    skiprows = 0
    if int(year) > 2006 and int(year) <= 2016:
        skiprows = 5
    dataframes.append(pd.read_csv('../data/yearly-data/' + str(year) + ".csv", skiprows = skiprows))


In [6]:
dataframes[0]['Site EUI (kBTU/sf)'] = dataframes[0].apply(lambda x: eui(x['E'], x['G'], x['O'],x['Building Gross Footage']),axis = 1)
dataframes[0]['Gross Area (sq ft)'] = dataframes[0]['Building Gross Footage']

In [7]:
dataframes[0]
# keep_columns = ['Address', 'Property Type', 'Gross Area (sq ft)', 'Site EUI (kBTU/sf)']
keep_columns = ['Address','Gross Area (sq ft)', 'Site EUI (kBTU/sf)']
for i in range(len(dataframes)):
    year = years[i]
    print(dataframes[i].columns)
    dataframes[i].columns = dataframes[i].columns.str.strip()


    dataframes[i] = dataframes[i].dropna(subset = ['Site EUI (kBTU/sf)'])
    dataframes[i] = dataframes[i][keep_columns]
    dataframes[i]['Address'] = dataframes[i]['Address'].str.lower()
    dataframes[i]['Address'] = dataframes[i]['Address'].str.strip()
    dataframes[i]['Address'] = dataframes[i]['Address'].str.replace(' ','')    

    dataframes[i] = dataframes[i].rename(columns = {'Site EUI (kBTU/sf)' : str(year) + 'EUI', 'Gross Area (sq ft)' : str(year) + 'Area'})
    area_ = str(year) + 'Area'
    eui_ = str(year) + 'EUI'

    dataframes[i][eui_] = dataframes[i][str(year) + "EUI"].apply(str)
    dataframes[i][area_] =(dataframes[i][str(year) + 'Area']).apply(str)
    


  
    dataframes[i][str(year) + 'EUI'] = dataframes[i][str(year) + 'EUI'].str.replace(',','')    
    dataframes[i][str(year) + 'Area'] = dataframes[i][str(year) + 'Area'].str.replace(',','')    

 
    
    dataframes[i][str(year) + 'EUI'] = pd.to_numeric(dataframes[i][str(year) + "EUI"],errors ='coerce').fillna(0).astype('int')
    dataframes[i][str(year) + 'Area'] = pd.to_numeric(dataframes[i][str(year) + 'Area'],errors ='coerce').fillna(0).astype('int')
    print(dataframes[i].columns)

Index(['Building Code', 'Address', 'Property Type', 'E', 'G', 'O',
       'Building Gross Footage', 'Site EUI (kBTU/sf)', 'Gross Area (sq ft)'],
      dtype='object')
Index(['Address', '2006Area', '2006EUI'], dtype='object')
Index(['Property Name', 'Reported', 'Property Type', 'Address', 'ZIP',
       'Gross Area (sq ft)', 'Site EUI (kBTU/sf)', 'Energy Star Score',
       'Energy Star Certified', 'Property Uses', 'Year Built',
       'GHG Emissions (MTCO2e)', 'GHG Intensity (kgCO2/sf)',
       ' Total Site Energy (kBTU) ', '% Electricity', '% Gas', '% Steam',
       'Water Intensity (gal/sf)', 'Onsite Solar (kWh)', 'User Submitted Info',
       'User Submitted Link', 'Tax Parcel'],
      dtype='object')
Index(['Address', '2015Area', '2015EUI'], dtype='object')
Index(['Property Name', 'Reported', 'Property Type', 'Address', 'ZIP',
       'Gross Area (sq ft)', 'Site EUI (kBTU/sf)', 'Energy Star Score',
       'Energy Star Certified', 'Property Uses', 'Year Built',
       'GHG Emissions (

In [8]:
#Join yearly data
merged_data = dataframes[0]
for i in range(1,len(dataframes)):
    merged_data = merged_data.merge(dataframes[i],on ='Address', how = 'left' )


In [9]:
def dist(a,b):
    return abs(a-b)


In [10]:

#Remove any outliers 
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

#Fill na
def fill(merged_data):


    for column_name in merged_data.columns[1:]:

        if column_name[4] =='A':
            continue
        year = column_name[:4]
        if int(year) <= 2006:
            continue
        
        def clean_and_mean(data):
            data = data[data['dist'] < 10000]
            data = remove_outlier(data, year + 'EUI')

            return data[year + 'EUI'].mean()
        def find_avg(df, area):
            df = pd.DataFrame(df)
            df['dist'] = 0
            df['dist'] = df.apply(lambda x: dist(area, x[year + 'Area']),axis = 1)

            return clean_and_mean(df.nsmallest(30,'dist'))
        
        idx = years.index(year)
        print(column_name)
        merged_data[column_name] = merged_data.apply(lambda x: find_avg(dataframes[idx], x[str(2006) + 'Area']) if pd.isna(x[column_name]) else x[column_name],axis = 1 )
    return merged_data  
     

In [26]:
merged_data = fill(merged_data)
data = pd.DataFrame()
for i in years:
    data[i] = merged_data[i + 'EUI']
y = data['2006']
X = data.loc[:, data.columns != '2006']
X = X.to_numpy()
y = y.to_numpy()


2015EUI
2016EUI
2017EUI
2018EUI
2019EUI
2020EUI
2021EUI


In [63]:
from sklearn.model_selection import train_test_split

trainX, validX, trainY, validY = train_test_split(X,y, test_size = 0.1)

In [83]:
def build_model():
  # Here we have sequentially: 1 input layer | 1 hidden layer | and 1 output layer
  model = keras.Sequential([
     
      layers.Dense(64, activation= 'relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1) # 1 because we want to return a single value with no transformations (no relu)
  ])

# initialize optimizer, loss, metrics
  model.compile(optimizer='adam',
                loss='mse',
                metrics=['mae'] # using mean absolute error
                )
  
  return model

In [84]:
model = build_model()
model.fit(trainX, trainY, epochs= 10, validation_data= (validX, validY))

Epoch 1/10
4/4 [==============================] - 0s 35ms/step - loss: 3132.3528 - mae: 51.5066 - val_loss: 2571.5435 - val_mae: 43.3665
Epoch 2/10
4/4 [==============================] - 0s 6ms/step - loss: 1377.2662 - mae: 34.1588 - val_loss: 1109.4172 - val_mae: 27.2463
Epoch 3/10
4/4 [==============================] - 0s 7ms/step - loss: 538.2068 - mae: 19.7278 - val_loss: 367.8112 - val_mae: 13.5525
Epoch 4/10
4/4 [==============================] - 0s 7ms/step - loss: 177.5373 - mae: 8.7050 - val_loss: 95.4248 - val_mae: 5.1592
Epoch 5/10
4/4 [==============================] - 0s 7ms/step - loss: 82.4709 - mae: 5.8104 - val_loss: 65.6649 - val_mae: 6.2749
Epoch 6/10
4/4 [==============================] - 0s 19ms/step - loss: 109.1714 - mae: 8.4093 - val_loss: 93.5871 - val_mae: 8.5301
Epoch 7/10
4/4 [==============================] - 0s 10ms/step - loss: 129.6045 - mae: 9.8444 - val_loss: 95.1998 - val_mae: 8.6166
Epoch 8/10
4/4 [==============================] - 0s 7ms/step - loss

In [85]:
predictions_data = pd.read_csv('../data/predictions.csv')

In [86]:
predictions_data = predictions_data.dropna()
for year in years[:]:
    predictions_data[year + 'EUI'] = pd.NA

predictions_data =  predictions_data.drop(['Code','ESPM Property Type','space@BU Property Type'], axis = 1)

In [87]:
predictions_data = fill(predictions_data)

2015EUI
2016EUI
2017EUI
2018EUI
2019EUI
2020EUI
2021EUI


In [88]:
predictions_data

,Address,2006Area,2006EUI,2015EUI,2016EUI,2017EUI,2018EUI,2019EUI,2020EUI,2021EUI
0,33 Harry Agganis Way,400268.0,<NA>,66.125000,51.909091,63.100000,56.307692,79.000000,72.916667,48.818182
1,815 Albany Street,102126.0,<NA>,89.148148,70.392857,69.809524,48.000000,47.678571,55.222222,59.545455
2,Peabody Hall (210 Riverway),58637.0,<NA>,73.185185,80.321429,64.071429,69.172414,56.500000,60.172414,53.629630
3,Riverway House (160-162 Riverway),31106.0,<NA>,67.034483,63.857143,54.586207,55.107143,60.500000,56.172414,50.296296
4,Campus Center and Student Residence (150 River...,57107.0,<NA>,63.807692,73.730769,65.814815,75.107143,67.285714,67.137931,48.172414
7,610 Commonwealth Avenue,170000.0,<NA>,71.344828,78.766667,77.066667,78.666667,83.966667,79.724138,61.107143


In [89]:
predictions_data

data = pd.DataFrame()

for i in years:
    data[i] = predictions_data[i + 'EUI']

testX = data.loc[:, data.columns != '2006']

testX = testX.to_numpy()


In [90]:
model.predict(testX)

array([[5.104158 ],
       [8.662122 ],
       [6.8759885],
       [5.9562964],
       [5.331755 ],
       [5.8242784]], dtype=float32)

In [91]:
data['2006'] = model.predict((testX)).flatten().tolist()
data['Address'] = predictions_data['Address']
data.to_csv('predicted_data.csv')

In [92]:
trainY

array([ 6, 11, 26, 12,  7,  5, 18, 32, 38,  1,  6,  4,  2,  2,  1,  6, 25,
        4,  4,  4, 24,  1,  4,  4,  5, 12,  5,  1,  5,  6,  2,  4,  4,  8,
        3,  3,  4,  4, 12,  4,  8,  3,  4,  2,  4,  3,  4,  5,  4, 11,  2,
       68,  6,  5,  2,  2,  6, 35,  6,  4,  3,  4,  3,  8,  1,  7,  3,  7,
       10,  5,  6,  6,  3,  5, 15, 11, 16, 11,  8, 40,  1,  4, 12,  4,  6,
       11,  4,  6, 12,  2,  6,  4,  7,  4,  4,  5,  9,  1, 11, 42,  4, 12,
        6,  0,  5,  4,  5, 33,  4,  5,  4,  3,  0,  4,  5,  3,  4,  3,  5,
        4,  3, 15,  5,  4,  4,  3])

In [100]:
model.predict(testX)

array([[5.104158 ],
       [8.662122 ],
       [6.8759885],
       [5.9562964],
       [5.331755 ],
       [5.8242784]], dtype=float32)

In [99]:
from sklearn.linear_model import LinearRegression


linear = LinearRegression()
linear.fit(trainX, trainY)
linear.score(validX, validY)



linear.predict(testX)

array([-2.66211912,  2.65321476, 12.34177303,  5.58720127, 10.75774499,
        5.38211527])